In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [7]:
dataset=dataset.drop("User ID" ,axis=1)

In [8]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [9]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [10]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 



grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
   
# fitting the model for grid search 
grid.fit(X_train_, y_train) 
 



Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [15]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test_) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9108102345415777


In [17]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 5 44]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93        85
           1       0.86      0.90      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.90       134
weighted avg       0.91      0.91      0.91       134



In [19]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.49039615846338536

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001403,0.000492,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001204,0.000406,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.023839,0.001613,0.004703,0.000866,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.727273,0.756757,0.809524,0.810811,0.882353,0.797343,0.053129,8
3,0.195761,0.002788,0.010515,0.000631,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.800000,0.850000,0.809524,0.926829,0.923077,0.861886,0.054175,1
4,0.022592,0.001539,0.004304,0.000399,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.687500,0.810811,0.780488,0.871795,0.923077,0.814734,0.080462,7
5,0.191451,0.007113,0.009584,0.001202,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.764706,0.871795,0.809524,0.900000,0.947368,0.858679,0.064796,3
6,0.001000,0.000632,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000902,0.000801,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.021241,0.000750,0.004312,0.000682,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.727273,0.810811,0.818182,0.842105,0.947368,0.829148,0.070681,5
9,0.206372,0.006522,0.009113,0.001165,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.764706,0.842105,0.809524,0.900000,0.947368,0.852741,0.064682,4


In [22]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Gender Male 0 or 1:"))

Age: 28
Salary: 20000
Gender Male 0 or 1: 1


In [23]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
